# NFIRS data input and processing

This series of 3 Jupyter notebooks consumes a NFIRS Fire/Hazmat data folder (one year, version 5.0 required), and outputs a GeoJSON file with coordinates from Google Maps Geocoding API.

This data is piped to Unity for visualization via Cesium.

## Part 1: NFIRS raw data intake, cleaning, and address extraction

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import csv
import os
import datetime
from collections import defaultdict
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
out_file_path = '2023_fires_in_tempe_loc1.csv'
folderpath = "./NFIRS_FIRES_2023" # unzip first

`codelookup.txt` is a lookup table that matches codes to their text descriptor. Important later

In [208]:
with open(f'{folderpath}/codelookup.txt', 'r') as f:
    lookup_table = pd.read_csv(f, sep="^")

lookup_table.head()

,fieldid,code_value,code_descr
0,ACT_TAK1,NaN,ACTIONS TAKEN 1
1,ACT_TAK1,00,"Action taken, other"
2,ACT_TAK1,1,Fire
3,ACT_TAK1,10,"Fire, other"
4,ACT_TAK1,11,Extinguish


`fireincident.txt` contains every actual fire reported to NFIRS that year.
(No purely medical incidents, false alarms, etc. Only actual fires)

In [ ]:
# get all fireincidents from given year in AZ
with open(f'{folderpath}/fireincident.txt','rb') as f:
    all_fires = pd.read_csv(f, sep="^", encoding="ISO-8859-1")

all_fires = all_fires[all_fires['STATE'] == 'AZ']

C:\Users\avuon\AppData\Local\Temp\ipykernel_5500\546489513.py:2: DtypeWarning: Columns (2,14,15,16,17,25,38,77,78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  all_fires = pd.read_csv(f, sep="^", encoding="ISO-8859-1")


<class 'pandas.core.frame.DataFrame'>
Index: 16525 entries, 29817 to 46341
Data columns (total 63 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   INCIDENT_KEY  16525 non-null  object 
 1   STATE         16525 non-null  object 
 2   FDID          16525 non-null  object 
 3   INC_DATE      16525 non-null  int64  
 4   INC_NO        16525 non-null  int64  
 5   EXP_NO        16525 non-null  int64  
 6   VERSION       16525 non-null  float64
 7   NUM_UNIT      14091 non-null  float64
 8   NOT_RES       9134 non-null   object 
 9   BLDG_INVOL    3309 non-null   float64
 10  ACRES_BURN    1492 non-null   float64
 11  LESS_1ACRE    7000 non-null   object 
 12  ON_SITE_M1    12056 non-null  object 
 13  MAT_STOR1     8808 non-null   object 
 14  ON_SITE_M2    24 non-null     object 
 15  MAT_STOR2     24 non-null     object 
 16  ON_SITE_M3    9 non-null      object 
 17  MAT_STOR3     9 non-null      object 
 18  AREA_ORIG     16525 non-nul

Create another lookup table that matches database column names to their long English names.

In [ ]:
# match up column shortnames to their longnames
col_names_long = ["Fire Dept. State", "Fire Dept. ID", "Incident Date", "Incident Number", "Exposure Number", "NFIRS Data Version", "Number of Residential Units", "Not Residential Flag", "Number of Buildings Involved", "Acres Burned", "Less Than One Acre", "On Site Materials #1", "Material Storage Use #1", "On Site Materials #2", "Material Storage Use #2", "On Site Materials #3", "Material Storage Use #3", "Area of Origin", "Heat Source", "Item First Ignited", "Confined To Origin", "Type of Material", "Cause of Ignition", "Factors Contributing To Ignition #1", "Factors Contributing To Ignition #2", "Human Factors #1", "Human Factors #2", "Human Factors #3", "Human Factors #4", "Human Factors #5", "Human Factors #6", "Human Factors #7", "Human Factors #8", "Age of Person", "Sex of Person", "Equipment Involved", "Suppression Factors #1", "Suppression Factors #2", "Suppression Factors #3", "Mobile Property Involved", "Mobile Property Type", "Mobile Property Make",
                  "Mobile Property Model", "Mobile Property Year", "Mobile Property License Plate", "Mobile Property State", "Mobile Property VIN Number", "Equipment Brand", "Equipment Model", "Equipment Serial Number", "Equipment Year", "Equipment Power", "Equipment Portability", "Fire Spread", "Structure Type", "Structure Status", "Building Height: Stories Above Grade", "Building Height: Stories Below Grade", "Building Length", "Building Width", "Total Square Feet", "Fire Origin (starting story)", "Number of Stories with Damage: Minor", "Number of Stories with Damage: Significant", "Number of Stories with Damage: Heavy", "Number of Stories with Damage: Extreme", "No Flame Spread/Same As First/Unknown", "Item Contributing Most to Spread", "Type Material Contributing Most To Spread", "Detector Presence", "Detector Type", "Detector Power", "Detector Operation", "Detector Effectiveness", "Detector Failure Reason", "AES Presence", "AES Type", "AES Operation", "Number of Sprinklers Operating", "AES Failure Reason",]

lookup_fire = dict(zip(all_fires.columns[1:],col_names_long))

all_fires = all_fires.loc[:, ~all_fires.columns.str.startswith(('MOB_','EQ_','ST_DAM_'))]

all_fires.info()

`incidentaddress.txt` contains the address of every NFIRS datapoint that year. Including non-fires.

This is where we can filter down to Tempe only.

In [ ]:
with open(f'{folderpath}/incidentaddress.txt','rb') as f:
    addresses = pd.read_csv(f, sep="^", encoding="ISO-8859-1")

address_col_long_names = ["Fire Dept. State", "Fire Dept. ID", "Incident Date", "Incident Number", "Exposure Number", "Location Type", "Number or Milepost",
                          "Street Prefix", "Street or Highway Name", "Street Type", "Street Suffix", "Apartment Number", "City", "State", "Zip 5", "Zip 4", "Cross Street or Directions",]
lookup_addr = dict(zip(addresses.columns[1:],address_col_long_names))

addresses = addresses[addresses['STATE'] == 'AZ']
addresses = addresses[addresses['CITY'] == 'Tempe']
addresses = addresses.drop(['STREETSUF','APT_NO','ZIP4'],axis=1) # Data is usually not helpful
#TODO STREETSUF is necessary sometimes for geocoding errors, but causes other errors too
addresses.head()

In [ ]:
lookup_addr.update(lookup_fire) # merge the two lookup tables
lookup = lookup_addr #convenience rename

{'STATE': 'Fire Dept. State',
 'FDID': 'Fire Dept. ID',
 'INC_DATE': 'Incident Date',
 'INC_NO': 'Incident Number',
 'EXP_NO': 'Exposure Number',
 'LOC_TYPE': 'Location Type',
 'NUM_MILE': 'Number or Milepost',
 'STREET_PRE': 'Street Prefix',
 'STREETNAME': 'Street or Highway Name',
 'STREETTYPE': 'Street Type',
 'STREETSUF': 'Street Suffix',
 'APT_NO': 'Apartment Number',
 'CITY': 'City',
 'STATE_ID': 'State',
 'ZIP5': 'Zip 5',
 'ZIP4': 'Zip 4',
 'X_STREET': 'Cross Street or Directions',
 'VERSION': 'NFIRS Data Version',
 'NUM_UNIT': 'Number of Residential Units',
 'NOT_RES': 'Not Residential Flag',
 'BLDG_INVOL': 'Number of Buildings Involved',
 'ACRES_BURN': 'Acres Burned',
 'LESS_1ACRE': 'Less Than One Acre',
 'ON_SITE_M1': 'On Site Materials #1',
 'MAT_STOR1': 'Material Storage Use #1',
 'ON_SITE_M2': 'On Site Materials #2',
 'MAT_STOR2': 'Material Storage Use #2',
 'ON_SITE_M3': 'On Site Materials #3',
 'MAT_STOR3': 'Material Storage Use #3',
 'AREA_ORIG': 'Area of Origin',
 'HEA

Merge `fireincident` and `incidentaddress` together, dropping any fires which don't occur in Tempe, AZ

In [ ]:
all_fires = all_fires.drop(['STATE', 'FDID', 'INC_DATE', 'INC_NO', 'EXP_NO'], axis=1)
fires_in_tempe = addresses.merge(all_fires, on='INCIDENT_KEY', how='inner')
# remove decimal point from zipcode
fires_in_tempe['ZIP5'] = fires_in_tempe['ZIP5'].astype(int)

#coerce col dtypes to help plaintext formatting look better
fires_in_tempe = fires_in_tempe.apply(lambda x: x.astype('int',errors='ignore') if x.dtype == 'float64' else x)
fires_in_tempe = fires_in_tempe.apply(lambda x: x.astype('object',errors='ignore') if x.dtype == 'float64' else x)
fires_in_tempe.map(lambda x: x.rstrip('.0') if isinstance(x,str) else x) # failed attempt to prevent decimals rendering in plaintext
fires_in_tempe.head()

,INCIDENT_KEY,STATE,FDID,INC_DATE,INC_NO,EXP_NO,LOC_TYPE,NUM_MILE,STREET_PRE,STREETNAME,STREETTYPE,CITY,STATE_ID,ZIP5,X_STREET,VERSION,NUM_UNIT,NOT_RES,BLDG_INVOL,ACRES_BURN,LESS_1ACRE,ON_SITE_M1,MAT_STOR1,ON_SITE_M2,MAT_STOR2,ON_SITE_M3,MAT_STOR3,AREA_ORIG,HEAT_SOURC,FIRST_IGN,CONF_ORIG,TYPE_MAT,CAUSE_IGN,FACT_IGN_1,FACT_IGN_2,HUM_FAC_1,HUM_FAC_2,HUM_FAC_3,HUM_FAC_4,HUM_FAC_5,HUM_FAC_6,HUM_FAC_7,HUM_FAC_8,AGE,SEX,EQUIP_INV,SUP_FAC_1,SUP_FAC_2,SUP_FAC_3,FIRE_SPRD,STRUC_TYPE,STRUC_STAT,BLDG_ABOVE,BLDG_BELOW,BLDG_LGTH,BLDG_WIDTH,TOT_SQ_FT,FIRE_ORIG,FLAME_SPRD,ITEM_SPRD,MAT_SPRD,DETECTOR,DET_TYPE,DET_POWER,DET_OPERAT,DET_EFFECT,DET_FAIL,AES_PRES,AES_TYPE,AES_OPER,NO_SPR_OP,AES_FAIL
0,AZ_08052_01022023_23164_0,AZ,08052,1022023,23164,0,1,2145,E,WARNER,RD,Tempe,AZ,85284,NaN,5,NaN,N,NaN,NaN,NaN,240,N,NaN,NaN,NaN,NaN,00,13,25,NaN,41,2,20,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AZ_08052_01102023_231089_0,AZ,08052,1102023,231089,0,1,9631,S,LA ROSA,DR,Tempe,AZ,85284,NaN,5,1.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,10,76,NaN,NaN,2,00,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AZ_08052_05112023_2314733_0,AZ,08052,5112023,2314733,0,1,1475,W,ELLIOT,RD,Tempe,AZ,85284,NaN,5,0.0,Y,1.0,NaN,NaN,UUU,U,NaN,NaN,NaN,NaN,UU,UU,UU,NaN,NaN,U,UU,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AZ_08052_05132023_2315036_0,AZ,08052,5132023,2315036,0,1,401,W,ORION,ST,Tempe,AZ,85283,NaN,5,NaN,N,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,UU,UU,UU,NaN,NaN,U,UU,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AZ_08052_06142023_2318787_0,AZ,08052,6142023,2318787,0,1,1905,E,VELVET,DR,Tempe,AZ,85284,NaN,5,1.0,N,1.0,NaN,NaN,UUU,U,NaN,NaN,NaN,NaN,93,UU,UU,NaN,NaN,U,UU,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NNN,NNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


`LOC_TYPE` 1 and 3 represent building fires. 2 often is for vehicle fires / car-crash fires, which have crossroads instead of full addresses and are much harder to geocode.

Drop #2-type fires here.

In [ ]:
def parse_addr_1(row):
    if row['LOC_TYPE'] not in (1,3):
        return None
    addr = list(row.iloc[7:14])
    addr = [str(x) if pd.notna(x) else '' for x in addr]
    addr = [x.strip() for x in addr if x.strip() != '']
    return ' '.join(addr)

fires_in_tempe['_addr'] = fires_in_tempe.apply(lambda x: parse_addr_1(x), axis=1)

For each cell in the non-address columns, replace the ID with its corresponding text from the lookup table.

This way we get to see that ON_SITE_M1=531 actually means a fire with charcoal fuel. Easier to read later, but harder to store

In [ ]:
temp = pd.DataFrame()

def match_code_to_lookup_name(val,colname):
    if isinstance(val,float) and not pd.isna(val):
        val = int(val)
    a = lookup_table.loc[(lookup_table['fieldid'] == colname) & (lookup_table['code_value'] == str(val)), 'code_descr'].values
    return a

for col in fires_in_tempe.columns[17:]:
    # if col not in lookup:
        # continue
    temp[col] = fires_in_tempe[col].map(lambda x: match_code_to_lookup_name(x,col))

temp = temp.map(lambda x: x[0] if len(list(x))>0 else None)
for col in temp.columns:
    temp[col] = temp[col].map(lambda x: str(lookup[col] + ': ' + str(x) if not pd.isna(x) else x))

A lot of None values got turned to strings here. Fix those, and also deal with N/NN/NNN, which are generic codes for NFIRS missing data.

In [ ]:
temp.replace(['None', 'nan', 'NaN'], np.nan, inplace=True)
# temp
fires_in_tempe.update(temp)
fires_in_tempe.replace(["N", "NN", "NNN"], np.nan, inplace=True)

C:\Users\avuon\AppData\Local\Temp\ipykernel_5500\389832939.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  temp.replace(['None', 'nan', 'NaN'], np.nan, inplace=True)
C:\Users\avuon\AppData\Local\Temp\ipykernel_5500\389832939.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fires_in_tempe.replace(["N", "NN", "NNN"], np.nan, inplace=True)


Move the `_addr` column to the front, then drop a couple columns which have info we don't need.

In [ ]:
cols = fires_in_tempe.columns.tolist()
cols.insert(1,cols.pop())
fires_in_tempe = fires_in_tempe[cols]
fires_in_tempe.drop(['FLAME_SPRD','VERSION'],axis=1,inplace=True)

Coerce dtypes again, then change the mm-dd-yyyy date string to an actual timestamp

In [ ]:
fires_in_tempe = fires_in_tempe.convert_dtypes()
fires_in_tempe['INC_DATE'] = pd.to_datetime(fires_in_tempe['INC_DATE'], format='%m%d%Y', errors='coerce')

In [ ]:
# Verify data here to make sure column count and labels match
for col in fires_in_tempe.columns:
    print(col,fires_in_tempe[col].dtype,lookup.get(col))

INCIDENT_KEY string None
_addr string None
STATE string Fire Dept. State
FDID string Fire Dept. ID
INC_DATE datetime64[ns] Incident Date
INC_NO Int64 Incident Number
EXP_NO Int64 Exposure Number
LOC_TYPE Int32 Location Type
NUM_MILE string Number or Milepost
STREET_PRE string Street Prefix
STREETNAME string Street or Highway Name
STREETTYPE string Street Type
CITY string City
STATE_ID string State
ZIP5 Int32 Zip 5
X_STREET string Cross Street or Directions
NUM_UNIT Int64 Number of Residential Units
NOT_RES string Not Residential Flag
BLDG_INVOL Int64 Number of Buildings Involved
ACRES_BURN Int64 Acres Burned
LESS_1ACRE string Less Than One Acre
ON_SITE_M1 string On Site Materials #1
MAT_STOR1 string Material Storage Use #1
ON_SITE_M2 Int64 On Site Materials #2
MAT_STOR2 Int64 Material Storage Use #2
ON_SITE_M3 Int64 On Site Materials #3
MAT_STOR3 Int64 Material Storage Use #3
AREA_ORIG string Area of Origin
HEAT_SOURC string Heat Source
FIRST_IGN string Item First Ignited
CONF_ORIG str

Some of the fire-data columns have numeric data, like occupant age, number of stories above ground, etc.

We label them similarly to the categorical data columns, changing `NO_SPR_OP=2` to "Number of Sprinklers Operating: 2" for readability

In [222]:
numeric_cols = ["NUM_UNIT","NOT_RES","LESS_1ACRE","AGE","SEX","BLDG_INVOL","ACRES_BURN","BLDG_ABOVE","BLDG_BELOW","BLDG_LGTH","BLDG_WIDTH","TOT_SQ_FT","FIRE_ORIG","NO_SPR_OP"]

def label(val,colname):
    if not pd.isna(val):
        return f"{lookup[colname]}: {str(val)}"
    else:
        return val

for col in fires_in_tempe[numeric_cols].columns:
    fires_in_tempe[col] = fires_in_tempe[col].map(lambda val: label(val,col))

Check here for missed cols, data errors, and bad addresses

In [ ]:
fires_in_tempe.sample(10)

,INCIDENT_KEY,_addr,STATE,FDID,INC_DATE,INC_NO,EXP_NO,LOC_TYPE,NUM_MILE,STREET_PRE,STREETNAME,STREETTYPE,CITY,STATE_ID,ZIP5,X_STREET,NUM_UNIT,NOT_RES,BLDG_INVOL,ACRES_BURN,LESS_1ACRE,ON_SITE_M1,MAT_STOR1,ON_SITE_M2,MAT_STOR2,ON_SITE_M3,MAT_STOR3,AREA_ORIG,HEAT_SOURC,FIRST_IGN,CONF_ORIG,TYPE_MAT,CAUSE_IGN,FACT_IGN_1,FACT_IGN_2,HUM_FAC_1,HUM_FAC_2,HUM_FAC_3,HUM_FAC_4,HUM_FAC_5,HUM_FAC_6,HUM_FAC_7,HUM_FAC_8,AGE,SEX,EQUIP_INV,SUP_FAC_1,SUP_FAC_2,SUP_FAC_3,FIRE_SPRD,STRUC_TYPE,STRUC_STAT,BLDG_ABOVE,BLDG_BELOW,BLDG_LGTH,BLDG_WIDTH,TOT_SQ_FT,FIRE_ORIG,ITEM_SPRD,MAT_SPRD,DETECTOR,DET_TYPE,DET_POWER,DET_OPERAT,DET_EFFECT,DET_FAIL,AES_PRES,AES_TYPE,AES_OPER,NO_SPR_OP,AES_FAIL
25,AZ_08293_01112023_23941_0,1800 E LIBRA DR Tempe AZ 85283,AZ,08293,2023-11-01,23941,0,1,1800,E,LIBRA,DR,Tempe,AZ,85283,<NA>,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"Area of Origin: Engine area, running gear, whe...",Heat Source: Undetermined,Item First Ignited: Undetermined,<NA>,<NA>,Cause of Ignition: Unintentional,Factors Contributing To Ignition #1: Undetermined,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
323,AZ_08293_10282023_2328438_0,3207 S WILSON ST Tempe AZ 85282,AZ,08293,2023-10-28,2328438,0,1,3207,S,WILSON,ST,Tempe,AZ,85282,<NA>,NaN,<NA>,NaN,NaN,<NA>,On Site Materials #1: Personal & home products...,Material Storage Use #1: Bulk storage or wareh...,<NA>,<NA>,<NA>,<NA>,Area of Origin: Cargo/trunk area - all vehicles,Heat Source: Undetermined,Item First Ignited: Undetermined,<NA>,<NA>,Cause of Ignition: Unintentional,Factors Contributing To Ignition #1: Undetermined,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
303,AZ_08293_09302023_2325835_0,1350 W 14TH ST Tempe AZ 85281,AZ,08293,2023-09-30,2325835,0,3,1350,W,14TH,ST,Tempe,AZ,85281,<NA>,Number of Residential Units: 0.0,Not Residential Flag: Y,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Area of Origin: Other,Heat Source: Heat source: other,"Item First Ignited: Item First Ignited, Other",<NA>,Type of Material: Multiple types of material,Cause of Ignition: Unintentional,Factors Contributing To Ignition #1: Other fac...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
377,AZ_08293_12182023_2332972_0,738 S PERRY LN Tempe AZ 85281,AZ,08293,2023-12-18,2332972,0,1,738,S,PERRY,LN,Tempe,AZ,85281,<NA>,Number of Residential Units: 0.0,Not Residential Flag: Y,Number of Buildings Involved: 1.0,NaN,Less Than One Acre: Y,On Site Materials #1: Insulation,Material Storage Use #1: Repair or service,<NA>,<NA>,<NA>,<NA>,"Area of Origin: Storage area, other","Heat Source: Radiated, conducted heat from ope...",Item First Ignited: Undetermined,<NA>,Type of Material: Undetermined,Cause of Ignition: Unintentional,Factors Contributing To Ignition #1: Heat sour...,<NA>,<NA>,<NA>,Human Factors #3: Unattended or unsupervised p...,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,"Equipment Involved: Heater, excluding catalyti...",<NA>,<NA>,<NA>,Fire Spread: Confined to room of origin,Structure Type: Enclosed building,Structure Status: In normal use,Building Height: Stories Above Grade: 1.0,Building Height: Stories Below Grade: 0.0,NaN,NaN,Total Square Feet: 2500.0,Fire Origin (starting story): 1.0,<NA>,<NA>,Detector Presence: None Present,<NA>,<NA>,<NA>,<NA>,<NA>,AES Presence: None Present,<NA>,<NA>,NaN,<NA>
159,AZ_08293_05152023_2312404_0,<NA>,AZ,08293,2023-05-15,2312404,0,2,<NA>,<NA>,A101,<NA>,Tempe,AZ,85281,RIO SALADO PW NB S,NaN,Not Residential Flag: Y,Number of Buildings Involved: 0.0,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"Area of Origin: Highway, parking lot, street: ...","Heat Source: Heat from powered equipment, other",Item First Ignited: Tire,<NA>,

For each column after the address/location data, concatenate them into a `_desc` column that has everything in one cell.

This is a temp solution to render all the data in Unity easily.

In [224]:
# #start row 16
import re
# fires_in_tempe.iloc[:,16:] = fires_in_tempe.iloc[:,16:].astype(str)
# fires_in_tempe['_desc'] = fires_in_tempe.apply(lambda row: '\n'.join(row[16:].fillna('')), axis=1)
fires_in_tempe['_desc'] = fires_in_tempe.apply(lambda row: '\n'.join(row[16:].map(lambda val: '' if pd.isna(val) else str(val))), axis=1)
fires_in_tempe['_desc'] = fires_in_tempe['_desc'].map(lambda val: re.sub(r'\n+', '\n', val))
fires_in_tempe['_desc'] = fires_in_tempe['_desc'].map(lambda val: val.strip('\n'))

Move `_desc` to front of the table

In [225]:
cols = fires_in_tempe.columns.tolist()
cols.insert(2,cols.pop())
fires_in_tempe = fires_in_tempe[cols]

Sort by date, drop anything that doesn't have an address value, then reindex.

In [ ]:
fires_in_tempe = fires_in_tempe.sort_values('INC_DATE')
fires_in_tempe = fires_in_tempe[fires_in_tempe['_addr'].notna()]
fires_in_tempe.reset_index(inplace=True, drop=True)

Check data again:

In [228]:
fires_in_tempe.sample(5)

,INCIDENT_KEY,_addr,_desc,STATE,FDID,INC_DATE,INC_NO,EXP_NO,LOC_TYPE,NUM_MILE,STREET_PRE,STREETNAME,STREETTYPE,CITY,STATE_ID,ZIP5,X_STREET,NUM_UNIT,NOT_RES,BLDG_INVOL,ACRES_BURN,LESS_1ACRE,ON_SITE_M1,MAT_STOR1,ON_SITE_M2,MAT_STOR2,ON_SITE_M3,MAT_STOR3,AREA_ORIG,HEAT_SOURC,FIRST_IGN,CONF_ORIG,TYPE_MAT,CAUSE_IGN,FACT_IGN_1,FACT_IGN_2,HUM_FAC_1,HUM_FAC_2,HUM_FAC_3,HUM_FAC_4,HUM_FAC_5,HUM_FAC_6,HUM_FAC_7,HUM_FAC_8,AGE,SEX,EQUIP_INV,SUP_FAC_1,SUP_FAC_2,SUP_FAC_3,FIRE_SPRD,STRUC_TYPE,STRUC_STAT,BLDG_ABOVE,BLDG_BELOW,BLDG_LGTH,BLDG_WIDTH,TOT_SQ_FT,FIRE_ORIG,ITEM_SPRD,MAT_SPRD,DETECTOR,DET_TYPE,DET_POWER,DET_OPERAT,DET_EFFECT,DET_FAIL,AES_PRES,AES_TYPE,AES_OPER,NO_SPR_OP,AES_FAIL
194,AZ_08293_10252023_2328166_0,1028 E ORANGE ST Tempe AZ 85281,"Area of Origin: Engine area, running gear, whe...",AZ,08293,2023-10-25,2328166,0,1,1028,E,ORANGE,ST,Tempe,AZ,85281,<NA>,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"Area of Origin: Engine area, running gear, whe...",Heat Source: Undetermined,Item First Ignited: Undetermined,<NA>,<NA>,Cause of Ignition: Unintentional,Factors Contributing To Ignition #1: Undetermined,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
64,AZ_08293_05032023_2311214_0,999 E BASELINE RD Tempe AZ 85283,Number of Residential Units: 1.0\nNumber of Bu...,AZ,08293,2023-05-03,2311214,0,1,999,E,BASELINE,RD,Tempe,AZ,85283,<NA>,Number of Residential Units: 1.0,<NA>,Number of Buildings Involved: 1.0,NaN,Less Than One Acre: Y,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"Area of Origin: Exterior balcony, unenclosed p...",Heat Source: Undetermined,Item First Ignited: Undetermined,<NA>,Type of Material: Undetermined,Cause of Ignition: Cause undetermined after in...,Factors Contributing To Ignition #1: Undetermined,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,Equipment Involved: Undetermined,<NA>,<NA>,<NA>,Fire Spread: Confined to room of origin,Structure Type: Enclosed building,Structure Status: In normal use,Building Height: Stories Above Grade: 4.0,Building Height: Stories Below Grade: 0.0,NaN,NaN,Total Square Feet: 593.0,Fire Origin (starting story): 3.0,<NA>,<NA>,Detector Presence: Detectors Present,Detector Type: Smoke,Detector Power: Hardwire with battery,Detector Operation: Fire too small to operate,<NA>,<NA>,AES Presence: None Present,<NA>,<NA>,NaN,<NA>
82,AZ_08293_05302023_2313827_0,2304 E RANDALL DR Tempe AZ 85281,Number of Residential Units: 2.0\nArea of Orig...,AZ,08293,2023-05-30,2313827,0,1,2304,E,RANDALL,DR,Tempe,AZ,85281,<NA>,Number of Residential Units: 2.0,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"Area of Origin: Courtyard, patio, porch, terrace",Heat Source: Heat source: other,Item First Ignited: Appliance housing or casing,<NA>,Type of Material: LP gas,Cause of Ignition: Failure of equipment or hea...,Factors Contributing To Ignition #1: Heat sour...,<NA>,<NA>,<NA>,Human Factors #3: Unattended or unsupervised p...,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
140,AZ_08293_08082023_2320699_0,200 E CURRY RD Tempe AZ 85281,Number of Residential Units: 0.0\nNot Resident...,AZ,08293,2023-08-08,2320699,0,1,200,E,CURRY,RD,Tempe,AZ,85281,<NA>,Number of Residential Units: 0.0,Not Residential Flag: Y,NaN,Acres Burned: 0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"Area of Origin: Outside area, other",Heat Source: Undetermined,Item First Ignited: Undetermined,Confined To Origin: Fire Spread was Confined t...,Type of Material: Undetermined,Cause of Ignition: Cause under investigation,Factors Contributing To Ignition #1: Undetermined,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>
47,AZ_08293_04142023_239358_0,58 S RIVER DR Tempe AZ 85281,Area of Origin: Undetermined\

Export the ID, address, description, and incident date to CSV for part 2.

In [229]:
fires_in_tempe[['INCIDENT_KEY','_addr','_desc','INC_DATE']].to_csv(out_file_path, index=False)